# <center> Neighbourhood Cyclability Analysis </center>
### <center> SIDs: 480398611 and 480139690 </center>

<div style="background-color:lightcyan;">
<span style="color:black"><center> <h1> Python packages </h1></center></span>
</div>

In [ ]:
import psycopg2
import psycopg2.extras

import csv
import pprint
import numpy as np

import pandas as pd
import re
import shapefile
import matplotlib.pyplot as plt
import seaborn as sns
sn = sns

import requests

from sklearn.metrics import confusion_matrix

<div style="background-color:lightcyan;">
<span style="color:black"><center> <h1> Connect to the postgresql database </h1></center></span>
</div>

In [ ]:
def pgconnect():
    # please replace <your_unikey> and <your_SID> with your own details
    YOUR_UNIKEY = 'xibi6298'#'<your_unikey>'
    YOUR_PW     = '480139690'#'<your_SID>'
    try: 
        conn = psycopg2.connect(host='soit-db-pro-1.ucc.usyd.edu.au',
                                database='y19s1d2x01_'+YOUR_UNIKEY,
                                user='y19s1d2x01_'+YOUR_UNIKEY,
                                password=YOUR_PW) 
        print('connected')
    except Exception as e:
        print("unable to connect to the database")
        print(e)
    return conn

conn = pgconnect()

In [ ]:
def pgexec( conn, sqlcmd, args, msg, silent=False ):
   """ utility function to execute some SQL statement
       can take optional arguments to fill in (dictionary)
       error and transaction handling built-in """
   retval = False
   with conn:
      with conn.cursor() as cur:
         try:
            if args is None:
               cur.execute(sqlcmd)
            else:
               cur.execute(sqlcmd, args)
            if silent == False: 
                print("success: " + msg)
            retval = True
         except Exception as e:
            if silent == False: 
                print("db error: ")
                print(e)
   return retval

In [ ]:
def pgquery( conn, sqlcmd, args, silent=False ):
   """ utility function to execute some SQL query statement
       can take optional arguments to fill in (dictionary)
       will print out on screen the result set of the query
       error and transaction handling built-in """
   retval = False
   with conn:
      with conn.cursor() as cur:
         try:
            if args is None:
                cur.execute(sqlcmd)
            else:
                cur.execute(sqlcmd, args)
            if silent == False:
                for record in cur:
                    print(record)
            retval = True
         except Exception as e:
            if silent == False:
                print("db read error: ")
                print(e)
   return retval

In [ ]:
def pgquery( conn, sqlcmd, args=None, msg=False, returntype='tuple'):
    """ utility function to execute some SQL query statement
        it can take optional arguments (as a dictionary) to fill in for placeholders in the SQL
        will return the complete query result as return value - or in case of error: None
        error and transaction handling built-in (by using the 'with' clauses)"""
    retval = None
    
    with conn:
        cursortype = None if returntype != 'dict' else psycopg2.extras.RealDictCursor
        with conn.cursor(cursor_factory=cursortype) as cur:
            try:
                if args is None:
                    cur.execute(sqlcmd)
                else:
                    cur.execute(sqlcmd, args)
                if (cur.description != None ):
                    retval = cur.fetchall() # we use fetchall() as we expect only _small_ query results
                if msg != False:
                    print("success: " + msg)
            except psycopg2.DatabaseError as e:
                if e.pgcode != None and msg:
                    print("db read error: "+msg)
                    print(e)
            except Exception as e:
                print(e)
    return retval

<div style="background-color:lightcyan;">
<span style="color:black"><center> <h1> Import Datasets </h1></center></span>
</div>

In [3]:
data_BusinessStats = list(csv.DictReader(open('BusinessStats.csv')))
header_BusinessStats = next(csv.reader(open('BusinessStats.csv')))

data_BikeSharingPods = list(csv.DictReader(open('BikeSharingPods.csv')))
header_BikeSharingPods = next(csv.reader(open('BikeSharingPods.csv')))

data_CensusStats = list(csv.DictReader(open('CensusStats.csv')))
header_CensusStats = next(csv.reader(open('CensusStats.csv')))

data_Neighbourhoods = list(csv.DictReader(open('Neighbourhoods.csv')))
header_Neighbourhoods = next(csv.reader(open('Neighbourhoods.csv')))

<div style="background-color:lightcyan;">
<span style="color:black"><center> <h1> Data Cleaning </h1></center></span>
</div>

#### Business Stats

In [3]:


filename = "BusinessStats.csv"
data = pd.read_csv(filename)

for elem in header_BusinessStats:
    data[elem].fillna('0', inplace = True)

newfile = 'NewBS.csv'
data= pd.DataFrame(data)
data.to_csv(newfile,index = False)
data_BS = list(csv.DictReader(open('NewBS.csv')))

#### Bike Sharing Pods

In [4]:


filename = "BikeSharingPods.csv"
data = pd.read_csv(filename)

for elem in header_BikeSharingPods:
    data[elem].fillna('0', inplace = True)

newfile = 'NewBSP.csv'
data= pd.DataFrame(data)
data.to_csv(newfile,index = False)
data_BSP = list(csv.DictReader(open('NewBSP.csv')))

#### Census Stats

In [5]:
filename = "CensusStats.csv"
data = pd.read_csv(filename)

for elem in header_CensusStats:
    data[elem].fillna('0', inplace = True)

newfile = 'NewCS.csv'
data= pd.DataFrame(data)
data.to_csv(newfile,index = False)
data_CS = list(csv.DictReader(open('NewCS.csv')))

#### Neighbourhoods

In [6]:
filename = "Neighbourhoods.csv"
data = pd.read_csv(filename)

for elem in header_Neighbourhoods:
    data[elem].fillna('0', inplace = True)

newfile = 'NewN.csv'
data= pd.DataFrame(data)
data.to_csv(newfile,index = False)
data_N = list(csv.DictReader(open('NewN.csv')))

<div style="background-color:lightcyan;">
<span style="color:black"><center> <h1> Data Integration and Database Generation </h1></center></span>
</div>

In [10]:
data_BusinessStats = list(csv.DictReader(open('NewBS.csv')))
data_BikeSharingPods = list(csv.DictReader(open('NewBSP.csv')))
data_CensusStats = list(csv.DictReader(open('NewCS.csv')))
data_Neighbourhoods = list(csv.DictReader(open('NewN.csv')))
data_StatisticalAreas = list(csv.DictReader(open('StatisticalAreas.csv')))

#### Schema for StatisticsalAreas

In [11]:
# 1st: login to database
conn = pgconnect()

# if you want to reset the table
pgexec (conn, "DROP TABLE IF EXISTS StatisticalAreas", None, "Reset Table StatisticalAreas")

# 2nd: ensure that the schema is in place

StatisticalAreas_schema = """CREATE TABLE IF NOT EXISTS StatisticalAreas (
                         area_id    INTEGER PRIMARY KEY,
                         area_name     VARCHAR(50),
                         parent_area_id  INTEGER
                        )"""
pgexec (conn, StatisticalAreas_schema, None, "Create Table StatisticalAreas")

# 3nd: load data
# IMPORTANT: make sure the header line of CSV is without spaces!
insert_stmt = """INSERT INTO StatisticalAreas(area_id,area_name,parent_area_id) VALUES (%(area_id)s, %(area_name)s, %(parent_area_id)s)"""
for row in data_StatisticalAreas:
    pgexec (conn, insert_stmt, row, "row inserted")
    
# cleaned

connected
success: Reset Table StatisticalAreas
success: Create Table StatisticalAreas
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: ro

#### Schema for Neighbourhoods

In [12]:
# 1st: login to database
conn = pgconnect()

# if you want to reset the table
pgexec (conn, "DROP TABLE IF EXISTS Neighbourhoods", None, "Reset Table Neighbourhoods")

# 2nd: ensure that the schema is in place

Neighbourhoods_schema = """CREATE TABLE IF NOT EXISTS Neighbourhoods(
                            area_id     INTEGER PRIMARY KEY,
                            area_name   VARCHAR(50),
                            land_area   NUMERIC,
                            population  NUMERIC,
                            number_of_dwellings    NUMERIC,
                            number_of_businesses  NUMERIC
                        )"""
pgexec (conn, Neighbourhoods_schema, None, "Create Table Neighbourhoods")

# 3nd: load data
# IMPORTANT: make sure the header line of CSV is without spaces!
insert_stmt = """INSERT INTO Neighbourhoods(area_id,area_name,land_area,population,number_of_dwellings,number_of_businesses) 
                    VALUES (%(area_id)s, %(area_name)s, %(land_area)s, %(population)s, %(number_of_dwellings)s, %(number_of_businesses)s)"""
for row in data_Neighbourhoods:
    pgexec (conn, insert_stmt, row, "row inserted")

#cleaned

connected
success: Reset Table Neighbourhoods
success: Create Table Neighbourhoods
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row in

#### Schema for BusinessStats

In [13]:
# 1st: login to database
conn = pgconnect()

# if you want to reset the table
pgexec (conn, "DROP TABLE IF EXISTS BusinessStats", None, "Reset Table BusinessStats")

# 2nd: ensure that the schema is in place

BusinessStats_schema = """CREATE TABLE IF NOT EXISTS BusinessStats (
                         area_id    INTEGER PRIMARY KEY,
                         num_businesses    NUMERIC,
                         retail_trade      NUMERIC,
                         accommodation_and_food_services   NUMERIC,
                         health_care_and_social_assistance   NUMERIC,
                         education_and_training   NUMERIC,
                         arts_and_recreation_services   NUMERIC
                        )"""
pgexec (conn, BusinessStats_schema, None, "Create Table BusinessStats")

# 3nd: load data
# IMPORTANT: make sure the header line of CSV is without spaces!
insert_stmt = """INSERT INTO BusinessStats(area_id,num_businesses,retail_trade,accommodation_and_food_services,health_care_and_social_assistance,education_and_training,arts_and_recreation_services) 
                VALUES (%(area_id)s, %(num_businesses)s, %(retail_trade)s , %(accommodation_and_food_services)s,%(health_care_and_social_assistance)s, %(education_and_training)s, %(arts_and_recreation_services)s)"""
for row in data_BusinessStats:
    pgexec (conn, insert_stmt, row, "row inserted")
    
#cleaned

connected
success: Reset Table BusinessStats
success: Create Table BusinessStats
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inse

#### Schema for CensusStats

In [14]:
# 1st: login to database
conn = pgconnect()

# if you want to reset the table
pgexec (conn, "DROP TABLE IF EXISTS CensusStats", None, "Reset Table CensusStats")

# 2nd: ensure that the schema is in place

CensusStats_schema = """CREATE TABLE IF NOT EXISTS CensusStats (
                         area_id    INTEGER PRIMARY KEY,
                         median_annual_household_income    NUMERIC,
                         avg_monthly_rent  NUMERIC
                        )"""
pgexec (conn, CensusStats_schema, None, "Create Table CensusStats")

# 3nd: load data
# IMPORTANT: make sure the header line of CSV is without spaces!
insert_stmt = """INSERT INTO CensusStats(area_id,median_annual_household_income,avg_monthly_rent) VALUES (%(area_id)s, %(median_annual_household_income)s, %(avg_monthly_rent)s)"""
for row in data_CensusStats:
    pgexec (conn, insert_stmt, row, "row inserted")
    

#clean

connected
success: Reset Table CensusStats
success: Create Table CensusStats
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted
success: row inserted

#### Schema for NeighbourhoodsWithBoundaries using the SA2 shapefile

In [15]:

sf = shapefile.Reader("SA2_2016_AUST.shp", encoding="iso-8859-1")

In [16]:
NeighbourhoodsWithBoundaries_schema =  """CREATE TABLE IF NOT EXISTS NeighbourhoodsWithBoundaries(
                            area_name     VARCHAR(50),
                            boundaries GEOMETRY(Polygon,4326)
                        )"""
                          

pgquery(conn, "DROP TABLE NeighbourhoodsWithBoundaries", msg="cleared old table")
pgquery(conn, NeighbourhoodsWithBoundaries_schema, msg="created NeighbourhoodsWithBoundaries_schema table")

success: cleared old table
success: created NeighbourhoodsWithBoundaries_schema table


In [17]:
import re

insert_stmt = """INSERT INTO NeighbourhoodsWithBoundaries VALUES ( %(area_name)s, ST_GEOMFROMTEXT(%(boundaries)s, 4326) )"""

shapes = sf.shapes()
records= sf.records()

row = {}
for i in range(0, len(shapes)):
    record = sf.record(i)
    shape  = sf.shape(i)
    row['area_name']=record[2]
    if record[8] == 'Greater Sydney':
        row['boundaries']="POLYGON(("
        i=0
        for x, y in shape.points:
            row['boundaries']+="%s %s," % (x,y)
       # check for start of a new polygon part
            i += 1
            if i in shape.parts:
                row['boundaries']= re.sub(",$", "),(", row['boundaries'])
    # properly end the polygon string
        row['boundaries'] = re.sub(",$", "))", row['boundaries'])
#     print(row['boundaries'])
    # finally: insert new row into the table
        pgquery(conn, insert_stmt, args=row, msg="inserted "+str(record[2]))
    # prepare the polygon data
    # this is a bit complex with our dataset as it has complex polygons, some with multiple parts...

success: inserted Avoca Beach - Copacabana
success: inserted Box Head - MacMasters Beach
success: inserted Calga - Kulnura
success: inserted Erina - Green Point
success: inserted Gosford - Springfield
success: inserted Kariong
success: inserted Kincumber - Picketts Valley
success: inserted Narara
success: inserted Niagara Park - Lisarow
success: inserted Point Clare - Koolewong
success: inserted Saratoga - Davistown
success: inserted Terrigal - North Avoca
success: inserted Umina - Booker Bay - Patonga
success: inserted Wamberal - Forresters Beach
success: inserted Woy Woy - Blackwall
success: inserted Wyoming
success: inserted Bateau Bay - Killarney Vale
success: inserted Blue Haven - San Remo
success: inserted Budgewoi - Buff Point - Halekulani
success: inserted Chittaway Bay - Tumbi Umbi
success: inserted Gorokan - Kanwal - Charmhaven
success: inserted Jilliby - Yarramalong
success: inserted Lake Munmorah - Mannering Park
success: inserted Ourimbah - Fountaindale
success: inserted S

success: inserted Blue Mountains - South
success: inserted Cambridge Park
success: inserted Castlereagh - Cranebrook
success: inserted Emu Plains - Leonay
success: inserted Glenmore Park - Regentville
success: inserted Jamisontown - South Penrith
success: inserted Kingswood - Werrington
success: inserted Mulgoa - Luddenham - Orchard Hills
success: inserted Penrith
success: inserted Warragamba - Silverdale
success: inserted Richmond - Clarendon
success: inserted Windsor - Bligh Park
success: inserted Yarramundi - Londonderry
success: inserted Erskine Park
success: inserted St Clair
success: inserted Colyton - Oxley Park
success: inserted St Marys - North St Marys
success: inserted Homebush Bay - Silverwater
success: inserted Rookwood Cemetery
success: inserted Auburn - Central
success: inserted Auburn - North
success: inserted Auburn - South
success: inserted Berala
success: inserted Lidcombe
success: inserted Regents Park
success: inserted Carlingford
success: inserted Ermington - Ryda

#### Schema for Bike sharing pods

In [18]:
#data_BikeSharingPods = list(csv.DictReader(open('BikeSharingPods.csv')))
data_BSP = list(csv.DictReader(open('NewBSP.csv')))
newfile = []

for j in range(0,len(data_BSP)):
    temp_dic = {}
    row = data_BSP[j]
    lat = row['latitude']
    lon = row['longitude']
#     point = "("+str(lat)+" "+str(lon)+")"
#     point = "POINT"+"%s %s," % (lat,lon)
    point = "POINT"+"("+str(lon)+" "+str(lat)+")"
# ST_GeomFromText('POINT(-0.1257 51.508)',4326)

    temp_dic['station_id'] = row['station_id']
    temp_dic['name']=row['name']
    temp_dic['num_bikes']=row['num_bikes']
    temp_dic['num_scooters']=row['num_scooters']
    temp_dic['point']=point
    temp_dic['description']=row['description']
    newfile.append(temp_dic)   
    
import pandas as pd
fileName = 'NewNeighbourhoods.csv'
number = 1
data= pd.DataFrame(newfile)
data.to_csv(fileName,index = False)
data_BikeSharingPods = list(csv.DictReader(open('NewNeighbourhoods.csv')))



In [1]:
# 1st: login to database
conn = pgconnect()

# if you want to reset the table
pgexec (conn, "DROP TABLE IF EXISTS BikeSharingPods", None, "Reset Table BikeSharingPods")

# 2nd: ensure that the schema is in place

BikeSharingPods_schema = """CREATE TABLE IF NOT EXISTS BikeSharingPods (
                         description   VARCHAR(500),
                         name     VARCHAR(100) ,
                         num_bikes  NUMERIC,
                         num_scooters  NUMERIC,
                         point GEOMETRY(Point, 4326) ,
                         station_id   NUMERIC PRIMARY KEY
                        )"""
pgexec (conn, BikeSharingPods_schema, None, "Create Table BikeSharingPods")

# 3nd: load data
# IMPORTANT: make sure the header line of CSV is without spaces!
insert_stmt = """INSERT INTO BikeSharingPods VALUES (%(description)s,%(name)s,%(num_bikes)s,%(num_scooters)s,ST_GEOMFROMTEXT(%(point)s, 4326),%(station_id)s)"""

for row in data_BikeSharingPods:
    temp_row = {}
    for i in row:
        temp_row[i] = row[i]
    pgquery(conn, insert_stmt, args=temp_row, msg="inserted ")

    
#cleaned

NameError: name 'pgconnect' is not defined

<div style="background-color:lightcyan;">
<span style="color:black"><center> <h1> Cyclability Analysis </h1></center></span>
</div>

#### Schema for bike_pods count of each area using Spatial Join

In [ ]:
query = """
select area_id, area_name,count(C.station_id) as bikepods_count from neighbourhoodswithboundaries B left join BikeSharingPods C on (ST_Contains(B.boundaries,C.point)) 
join statisticalareas using (area_name)
group by area_name, area_id
order by count(C.station_id) desc
"""
result= pgquery(conn, query,returntype='dict')
pprint.pprint(result)

[RealDictRow([('area_id', 117031336),
              ('area_name', 'Surry Hills'),
              ('bikepods_count', 45)]),
 RealDictRow([('area_id', 117031332),
              ('area_name', 'Newtown - Camperdown - Darlington'),
              ('bikepods_count', 41)]),
 RealDictRow([('area_id', 117031333),
              ('area_name', 'Potts Point - Woolloomooloo'),
              ('bikepods_count', 39)]),
 RealDictRow([('area_id', 117031335),
              ('area_name', 'Redfern - Chippendale'),
              ('bikepods_count', 29)]),
 RealDictRow([('area_id', 117031331),
              ('area_name', 'Glebe - Forest Lodge'),
              ('bikepods_count', 21)]),
 RealDictRow([('area_id', 121041417),
              ('area_name', 'North Sydney - Lavender Bay'),
              ('bikepods_count', 21)]),
 RealDictRow([('area_id', 121041416),
              ('area_name', 'Neutral Bay - Kirribilli'),
              ('bikepods_count', 19)]),
 RealDictRow([('area_id', 117031334),
              ('area_n

              ('bikepods_count', 0)]),
 RealDictRow([('area_id', 127011594),
              ('area_name', 'Green Valley'),
              ('bikepods_count', 0)]),
 RealDictRow([('area_id', 120011383),
              ('area_name', 'Concord - Mortlake - Cabarita'),
              ('bikepods_count', 0)]),
 RealDictRow([('area_id', 119041381),
              ('area_name', 'Rockdale - Banksia'),
              ('bikepods_count', 0)]),
 RealDictRow([('area_id', 120011386),
              ('area_name', 'Five Dock - Abbotsford'),
              ('bikepods_count', 0)]),
 RealDictRow([('area_id', 102011033),
              ('area_name', 'Kariong'),
              ('bikepods_count', 0)]),
 RealDictRow([('area_id', 124051469),
              ('area_name', 'Erskine Park'),
              ('bikepods_count', 0)]),
 RealDictRow([('area_id', 119011571),
              ('area_name', 'Bankstown - North'),
              ('bikepods_count', 0)]),
 RealDictRow([('area_id', 124031463),
              ('area_name', 'Mulgoa 

In [ ]:
conn = pgconnect()

# if you want to reset the table
pgexec (conn, "DROP TABLE IF EXISTS bikepods_count", None, "Reset Table bikepods_count")

# 2nd: ensure that the schema is in place
bikepods_count_schema = """CREATE TABLE IF NOT EXISTS bikepods_count (
                         area_id   INTEGER,
                         area_name VARCHAR(50),
                         bikepods_count INTEGER
                        )"""
pgexec (conn, bikepods_count_schema, None, "Create Table bikepods_count")

# 3nd: load data
# IMPORTANT: make sure the header line of CSV is without spaces!
insert_stmt = """INSERT INTO bikepods_count VALUES (%(area_id)s,%(area_name)s,%(bikepods_count)s)"""

for row in result:
    temp_row = {}
    for i in row:
        temp_row[i] = row[i]
    pgquery(conn, insert_stmt, args=temp_row, msg="inserted ")

connected
success: Reset Table bikepods_count
success: Create Table bikepods_count
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
succe

#### Schema for service_score 
- Calculate the service score

In [ ]:
query = """
        with ser as(
        select avg(retail_trade)/avg(num_businesses) as retail_wei,
        avg(accommodation_and_food_services)/avg(num_businesses) as acc_wei,
        avg(health_care_and_social_assistance)/avg(num_businesses) as health_wei,
        avg(education_and_training)/avg(num_businesses) as edu_wei,
        avg(arts_and_recreation_services)/avg(num_businesses) as art_wei
            from BusinessStats
             )
      select (retail_trade*ser.retail_wei+
             accommodation_and_food_services*ser.acc_wei+
             health_care_and_social_assistance*ser.health_wei+
             education_and_training*ser.edu_wei+
             arts_and_recreation_services*ser.art_wei)/rowsum
            as service_score, area_id
     from (select area_id,retail_trade,accommodation_and_food_services,
		   health_care_and_social_assistance,education_and_training,arts_and_recreation_services,
		   education_and_training+arts_and_recreation_services+
		   retail_trade+
		   accommodation_and_food_services+
		   health_care_and_social_assistance+
		   education_and_training+
		   arts_and_recreation_services as rowsum from BusinessStats) as tempquery,
		   ser
	where rowsum<>0	 
"""
result= pgquery(conn, query,returntype='dict')
pprint.pprint(result)


[RealDictRow([('service_score', Decimal('0.039931046996989518885')),
              ('area_id', 102011028)]),
 RealDictRow([('service_score', Decimal('0.034535922274436550919')),
              ('area_id', 102011029)]),
 RealDictRow([('service_score', Decimal('0.028462871345512907031')),
              ('area_id', 102011030)]),
 RealDictRow([('service_score', Decimal('0.047768351306516644764')),
              ('area_id', 102011031)]),
 RealDictRow([('service_score', Decimal('0.045653404532279685472')),
              ('area_id', 102011032)]),
 RealDictRow([('service_score', Decimal('0.037711957189104569628')),
              ('area_id', 102011033)]),
 RealDictRow([('service_score', Decimal('0.038117082532393320521')),
              ('area_id', 102011034)]),
 RealDictRow([('service_score', Decimal('0.032973979933093085635')),
              ('area_id', 102011035)]),
 RealDictRow([('service_score', Decimal('0.041018519114748030620')),
              ('area_id', 102011036)]),
 RealDictRow([('ser

In [ ]:
conn = pgconnect()

# if you want to reset the table
pgexec (conn, "DROP TABLE IF EXISTS service_score", None, "Reset Table service_score")

# 2nd: ensure that the schema is in place
service_score_schema = """CREATE TABLE IF NOT EXISTS service_score (
                         area_id   INTEGER PRIMARY KEY,
                         service_score NUMERIC
                        )"""
pgexec (conn, service_score_schema, None, "Create Table service_score")

# 3nd: load data
# IMPORTANT: make sure the header line of CSV is without spaces!
insert_stmt = """INSERT INTO service_score VALUES (%(area_id)s,%(service_score)s)"""

for row in result:
    temp_row = {}
    for i in row:
        temp_row[i] = row[i]
    pgquery(conn, insert_stmt, args=temp_row, msg="inserted ")

connected
success: Reset Table service_score
success: Create Table service_score
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success

#### Schema for measurement
- store all the z_scores

In [ ]:
query = """
with population_me as
    (select avg(population) as mean,
            stddev(population) as sd
    from Neighbourhoods),
    
   dwellings_me as
    (select avg(number_of_dwellings) as mean,
            stddev(number_of_dwellings) as sd
    from Neighbourhoods),
    
    ser_me as
    (select avg(service_score) as mean,
            stddev(service_score) as sd
    from service_score),
    
    bike_me as
    (select avg(bikepods_count) as mean,
            stddev(bikepods_count) as sd
    from bikepods_count),
    
    bsa_me as
    (select avg(bsascore) as mean,
            stddev(bsascore) as sd
    from BSAscore)
    
select area_id,area_name,
            abs(population - population_me.mean) / population_me.sd as z_score_population,
            abs(number_of_dwellings - dwellings_me.mean) / dwellings_me.sd as z_score_dwellings,
            abs(service_score - ser_me.mean) / ser_me.sd as z_score_service,
            abs(bikepods_count - bike_me.mean) / bike_me.sd as z_score_bike,
            abs(bsascore - bsa_me.mean) / bsa_me.sd as z_score_bsa
        from Neighbourhoods natural join 
            BusinessStats natural join 
            service_score natural join
            BSAscore natural join 
            bikepods_count,
            population_me,
            dwellings_me, 
            ser_me,
            bike_me,
            bsa_me;

"""
result= pgquery(conn, query,returntype='dict')
pprint.pprint(result)


[RealDictRow([('area_id', 125011582),
              ('area_name', 'Auburn - Central'),
              ('z_score_population', Decimal('0.27051685156201168837')),
              ('z_score_dwellings', Decimal('0.25090801700579439186')),
              ('z_score_service',
               Decimal('1.139406339589460841250546364828034914164400')),
              ('z_score_bike', Decimal('0.25233448228763915351')),
              ('z_score_bsa',
               Decimal('0.1730165126721970316862536543288554222088'))]),
 RealDictRow([('area_id', 126021591),
              ('area_name', 'Ryde'),
              ('z_score_population', Decimal('1.7079744608704649')),
              ('z_score_dwellings', Decimal('1.9154735576713133')),
              ('z_score_service',
               Decimal('0.012260244916382649884838895169221360767077')),
              ('z_score_bike', Decimal('0.25233448228763915351')),
              ('z_score_bsa',
               Decimal('0.1730165126721970316862536543288554222088'))]),
 R

              ('z_score_dwellings', Decimal('0.61383622544684672397')),
              ('z_score_service',
               Decimal('0.637527023976600253404263483367099563864312')),
              ('z_score_bike', Decimal('0.25233448228763915351')),
              ('z_score_bsa',
               Decimal('0.1730165126721970316862536543288554222088'))]),
 RealDictRow([('area_id', 127011595),
              ('area_name', 'Hinchinbrook'),
              ('z_score_population', Decimal('0.40207604573775261824')),
              ('z_score_dwellings', Decimal('0.72034290067369068827')),
              ('z_score_service',
               Decimal('1.271742747601209326947161739475446456538580')),
              ('z_score_bike', Decimal('0.25233448228763915351')),
              ('z_score_bsa',
               Decimal('0.1730165126721970316862536543288554222088'))]),
 RealDictRow([('area_id', 102021047),
              ('area_name', 'Chittaway Bay - Tumbi Umbi'),
              ('z_score_population', Decimal('0.0

              ('z_score_dwellings', Decimal('0.19928406399506707065')),
              ('z_score_service',
               Decimal('0.283772795435344664829623595053711194450464')),
              ('z_score_bike', Decimal('0.25233448228763915351')),
              ('z_score_bsa',
               Decimal('0.1730165126721970316862536543288554222088'))]),
 RealDictRow([('area_id', 120031390),
              ('area_name', 'Ashfield'),
              ('z_score_population', Decimal('1.2300928860694230')),
              ('z_score_dwellings', Decimal('1.5292749832143119')),
              ('z_score_service',
               Decimal('0.028169480916042532904492956417985235313116')),
              ('z_score_bike', Decimal('0.49705794770813373859')),
              ('z_score_bsa',
               Decimal('0.1730165126721970316862536543288554222088'))]),
 RealDictRow([('area_id', 124051580),
              ('area_name', 'Colyton - Oxley Park'),
              ('z_score_population', Decimal('0.5558624035859602544

              ('z_score_bsa',
               Decimal('0.1730165126721970316862536543288554222088'))]),
 RealDictRow([('area_id', 120011384),
              ('area_name', 'Concord West - North Strathfield'),
              ('z_score_population', Decimal('1.3135950654960944')),
              ('z_score_dwellings', Decimal('1.7516859814821331')),
              ('z_score_service',
               Decimal('0.293809624257862602481880990222541789562032')),
              ('z_score_bike', Decimal('0.25233448228763915351')),
              ('z_score_bsa',
               Decimal('0.1730165126721970316862536543288554222088'))]),
 RealDictRow([('area_id', 126021501),
              ('area_name', 'North Ryde - East Ryde'),
              ('z_score_population', Decimal('0.44185714491666913902')),
              ('z_score_dwellings', Decimal('0.45586624076165378534')),
              ('z_score_service',
               Decimal('0.176864940555819810088711077747562388222225')),
              ('z_score_bike', Deci

              ('z_score_bsa',
               Decimal('0.1730165126721970316862536543288554222088'))]),
 RealDictRow([('area_id', 119021573),
              ('area_name', 'Lakemba'),
              ('z_score_population', Decimal('0.23454863728842262965')),
              ('z_score_dwellings', Decimal('0.10396506473742940513')),
              ('z_score_service',
               Decimal('1.026800457237645507503820318152797403350856')),
              ('z_score_bike', Decimal('0.25233448228763915351')),
              ('z_score_bsa',
               Decimal('0.1730165126721970316862536543288554222088'))]),
 RealDictRow([('area_id', 119031374),
              ('area_name', 'South Hurstville - Blakehurst'),
              ('z_score_population', Decimal('1.1421423409198695')),
              ('z_score_dwellings', Decimal('1.0665527051279394')),
              ('z_score_service',
               Decimal('1.144939784397211095265377797275472030523255')),
              ('z_score_bike', Decimal('0.25233448228

              ('area_name', 'Springwood - Winmalee'),
              ('z_score_population', Decimal('0.71942050108256488772')),
              ('z_score_dwellings', Decimal('1.1770869437036976')),
              ('z_score_service',
               Decimal('1.114175549003716336295700730892574165749323')),
              ('z_score_bike', Decimal('0.25233448228763915351')),
              ('z_score_bsa',
               Decimal('0.1730165126721970316862536543288554222088'))]),
 RealDictRow([('area_id', 123031446),
              ('area_name', 'Douglas Park - Appin'),
              ('z_score_population', Decimal('0.75985174602097947445')),
              ('z_score_dwellings', Decimal('0.76106604120160161579')),
              ('z_score_service',
               Decimal('1.905039818982276728513054333427476005443686')),
              ('z_score_bike', Decimal('0.25233448228763915351')),
              ('z_score_bsa',
               Decimal('0.1730165126721970316862536543288554222088'))]),
 RealDictRow([(

In [ ]:
conn = pgconnect()

# if you want to reset the table
pgexec (conn, "DROP TABLE IF EXISTS measurement", None, "Reset Table measurement")

# 2nd: ensure that the schema is in place
measurement_schema = """CREATE TABLE IF NOT EXISTS measurement (
                         area_id   INTEGER PRIMARY KEY,
                         area_name     VARCHAR(100) ,
                         z_score_population DECIMAl,
                         z_score_dwellings DECIMAl,
                         z_score_service DECIMAl,
                         z_score_bike DECIMAl,
                         z_score_bsa DECIMAl
                        )"""
pgexec (conn, measurement_schema, None, "Create Table measurement")

# 3nd: load data
# IMPORTANT: make sure the header line of CSV is without spaces!
insert_stmt = """INSERT INTO measurement VALUES (%(area_id)s,%(area_name)s,%(z_score_population)s,%(z_score_dwellings)s,%(z_score_service)s,%(z_score_bike)s,%(z_score_bsa)s)"""

for row in result:
    temp_row = {}
    for i in row:
        temp_row[i] = row[i]
    pgquery(conn, insert_stmt, args=temp_row, msg="inserted ")

connected
success: Reset Table measurement
success: Create Table measurement
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: in

In [ ]:
query = """
select area_id, area_name, (z_score_population+ z_score_dwellings+ z_score_service+ z_score_bike +z_score_bsa) as cyclability
from measurement
"""
result= pgquery(conn, query,returntype='dict')
pprint.pprint(result)

[RealDictRow([('area_id', 125011582),
              ('area_name', 'Auburn - Central'),
              ('cyclability',
               Decimal('2.086182203117103106676800019156890336373200'))]),
 RealDictRow([('area_id', 126021591),
              ('area_name', 'Ryde'),
              ('cyclability',
               Decimal('4.061059258417997035081092549498076782975877'))]),
 RealDictRow([('area_id', 121031410),
              ('area_name', 'St Ives'),
              ('cyclability',
               Decimal('2.198724753955572297611885203121367539612447'))]),
 RealDictRow([('area_id', 118021568),
              ('area_name', 'Maroubra - West'),
              ('cyclability',
               Decimal('2.074389924937053656766790798965068791843736'))]),
 RealDictRow([('area_id', 102011029),
              ('area_name', 'Box Head - MacMasters Beach'),
              ('cyclability',
               Decimal('3.067645807892745257573539031523424938528007'))]),
 RealDictRow([('area_id', 126021590),
             

 RealDictRow([('area_id', 124031457),
              ('area_name', 'Cambridge Park'),
              ('cyclability',
               Decimal('0.968213931793105361725877249382566616659264'))]),
 RealDictRow([('area_id', 120031390),
              ('area_name', 'Ashfield'),
              ('cyclability',
               Decimal('3.457611810580108203180746610746840657521916'))]),
 RealDictRow([('area_id', 124051580),
              ('area_name', 'Colyton - Oxley Park'),
              ('cyclability',
               Decimal('1.818864161077308764821977655228474756682835'))]),
 RealDictRow([('area_id', 120031575),
              ('area_name', 'Strathfield'),
              ('cyclability',
               Decimal('5.026974279649879363174872018887514214294466'))]),
 RealDictRow([('area_id', 102021052),
              ('area_name', 'Summerland Point - Gwandalan'),
              ('cyclability',
               Decimal('2.979122922266736258906873298862201046016287'))]),
 RealDictRow([('area_id', 120011383),
 

              ('cyclability',
               Decimal('0.694363092123953685897914070812478259113359'))]),
 RealDictRow([('area_id', 115041301),
              ('area_name', 'Pitt Town - McGraths Hill'),
              ('cyclability',
               Decimal('2.691833608599095528281006050977139549598447'))]),
 RealDictRow([('area_id', 121031408),
              ('area_name', 'Lindfield - Roseville'),
              ('cyclability',
               Decimal('3.624252075350550131401280482485645900502253'))]),
 RealDictRow([('area_id', 102011040),
              ('area_name', 'Umina - Booker Bay - Patonga'),
              ('cyclability',
               Decimal('3.654649244712449382869456862383583620569085'))]),
 RealDictRow([('area_id', 119041377),
              ('area_name', 'Kingsgrove (South) - Bardwell Park'),
              ('cyclability',
               Decimal('1.349526106663604875192969650010943418999780'))]),
 RealDictRow([('area_id', 116021309),
              ('area_name', 'Glenwood'),
    

               Decimal('2.180860077705909017047193589683912529900560'))]),
 RealDictRow([('area_id', 124011454),
              ('area_name', 'Springwood - Winmalee'),
              ('cyclability',
               Decimal('3.436033988749815009211954385221429587958123'))]),
 RealDictRow([('area_id', 123031446),
              ('area_name', 'Douglas Park - Appin'),
              ('cyclability',
               Decimal('3.851308601164694003949307987756331427652486'))]),
 RealDictRow([('area_id', 125041489),
              ('area_name', 'North Parramatta'),
              ('cyclability',
               Decimal('2.841906877539750849937687809092090037163131'))]),
 RealDictRow([('area_id', 121041413),
              ('area_name', 'Cremorne - Cammeray'),
              ('cyclability',
               Decimal('3.060295198793695143394317368558033531094303'))]),
 RealDictRow([('area_id', 119031371),
              ('area_name', 'Oatley - Hurstville Grove'),
              ('cyclability',
               Deci

#### Schema for cyclability

In [ ]:
conn = pgconnect()

# if you want to reset the table
pgexec (conn, "DROP TABLE IF EXISTS cyclability", None, "Reset Table cyclability")

# 2nd: ensure that the schema is in place
cyclability_schema = """CREATE TABLE IF NOT EXISTS cyclability (
                         area_id   INTEGER PRIMARY KEY,
                         area_name     VARCHAR(100),
                         cyclability  DECIMAl
                        )"""
pgexec (conn, cyclability_schema, None, "Create Table cyclability")

# 3nd: load data
# IMPORTANT: make sure the header line of CSV is without spaces!
insert_stmt = """INSERT INTO cyclability VALUES (%(area_id)s,%(area_name)s,%(cyclability)s)"""

for row in result:
    temp_row = {}
    for i in row:
        temp_row[i] = row[i]
    pgquery(conn, insert_stmt, args=temp_row, msg="inserted ")

connected
success: Reset Table cyclability
success: Create Table cyclability
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: inserted 
success: in

#### Create cyclability.csv
- Store cyclability scores

In [ ]:
import pandas as pd
fileName = 'cyclability.csv'
data= pd.DataFrame(result)
data.to_csv(fileName,index = False)
data_cyclability = list(csv.DictReader(open('cyclability.csv')))

### Results: The top ten neighbourhoods with the highest cyclability scores

In [ ]:
query = """
select *
from cyclability
order by cyclability
limit 10
"""
topTenresult= pgquery(conn, query,returntype='dict')
pprint.pprint(topTenresult)

[RealDictRow([('area_id', 124041467),
              ('area_name', 'Windsor - Bligh Park'),
              ('cyclability',
               Decimal('0.610885999123519522056050700130503435737572'))]),
 RealDictRow([('area_id', 102021053),
              ('area_name', 'The Entrance'),
              ('cyclability',
               Decimal('0.646651371640414921271914938518825363907892'))]),
 RealDictRow([('area_id', 128011531),
              ('area_name', 'Sylvania - Taren Point'),
              ('cyclability',
               Decimal('0.694363092123953685897914070812478259113359'))]),
 RealDictRow([('area_id', 118021565),
              ('area_name', 'Kingsford'),
              ('cyclability',
               Decimal('0.723655724368699944696684994186113531093457'))]),
 RealDictRow([('area_id', 119041380),
              ('area_name', 'Monterey - Brighton-le-Sands - Kyeemagh'),
              ('cyclability',
               Decimal('0.741250611117461263380699606507931225702666'))]),
 RealDictRow([('ar

### Heatmap: the distribution of cyclability scores

In [ ]:
import numpy as np
import pandas as pd
import re
import shapefile
import seaborn as sns
from matplotlib import pyplot as plt, patches
import os
import warnings

In [ ]:
def read_shapefile(sf):
    """
    Read a shapefile into a Pandas dataframe with a 'coords' 
    column holding the geometry information. This uses the pyshp
    package
    """
    fields = [x[0] for x in sf.fields][1:]
    records = [y[:] for y in sf.records()]
    shps = [s.points for s in sf.shapes()]
    df = pd.DataFrame(columns=fields, data=records)
    df = df.assign(coords=shps)
    return df

In [ ]:
sf = shapefile.Reader("SA2_2016_AUST.shp", encoding="iso-8859-1")
df = read_shapefile(sf)

In [ ]:
def plot_map_fill_multiples_ids_tone(sf, title, comuna,  
                                     print_id, color_ton, 
                                     bins, 
                                     x_lim = None, 
                                     y_lim = None, 
                                     figsize = (11,9)):
    '''
    Plot map with lim coordinates
    '''
        
    plt.figure(figsize = figsize)
    
    fig, ax = plt.subplots(figsize = figsize)
    
    
    fig.suptitle(title, fontsize=16)
    
    shapes = sf.shapes()
    records= sf.records()
    lineCount = 0
    for shape in sf.shapeRecords():
        record = sf.record(lineCount)            
        if record[8] == 'Greater Sydney':
            x = [i[0] for i in shape.shape.points[:]]
            y = [i[1] for i in shape.shape.points[:]]
            ax.plot(x, y, 'k')
            if lineCount == 1:
                break
        lineCount+=1   
            
    for id in comuna:
        shape_ex = sf.shape(id)
        x_lon = np.zeros((len(shape_ex.points),1))
        y_lat = np.zeros((len(shape_ex.points),1))
        for ip in range(len(shape_ex.points)):
            x_lon[ip] = shape_ex.points[ip][0]
            y_lat[ip] = shape_ex.points[ip][1]
        ax.fill(x_lon,y_lat, color_ton[comuna.index(id)])
        if print_id != False:
            x0 = np.mean(x_lon)
            y0 = np.mean(y_lat)
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", category=RuntimeWarning)
                foo = np.nanmean(y0, axis=0)
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", category=RuntimeWarning)
                foo = np.nanmean(x0, axis=0)
#             plt.text(x0, y0, id, fontsize=10)
    if (x_lim != None) & (y_lim != None):     
        plt.xlim(x_lim)
        plt.ylim(y_lim)

In [ ]:
def calc_color(data, color=None):
        if color   == 1: 
            color_sq =  ['#dadaebFF','#bcbddcF0','#9e9ac8F0','#807dbaF0','#6a51a3F0','#54278fF0']; 
            colors = 'Purples';
        elif color == 2: 
            color_sq = ['#c7e9b4','#7fcdbb','#41b6c4', '#1d91c0','#225ea8','#253494']; 
            colors = 'YlGnBu';
        elif color == 3:
            color_sq = ['#f7f7f7','#d9d9d9','#bdbdbd','#969696','#636363','#252525']; 
            colors = 'Greys';
        elif color == 9: 
            color_sq = ['#ff0000','#ff0000','#ff0000','#ff0000','#ff0000','#ff0000']
        else:            
            color_sq = ['#ffffd4','#fee391','#fec44f','#fe9929','#d95f0e','#993404']; 
            colors = 'YlOrBr';
        new_data, bins = pd.qcut(data, 6, retbins=True, labels=list(range(6)))
        color_ton = []
        for val in new_data:
            color_ton.append(color_sq[val]) 
        if color != 9:
            colors = sns.color_palette(colors, n_colors=6)
            sns.palplot(colors, 0.6);
            for i in range(6):
                print ("\n"+str(i+1)+': '+str(int(bins[i]))+
                       " => "+str(int(bins[i+1])-1), end =" ")
            print("\n\n   1   2   3   4   5   6")    
        return color_ton, bins;

In [ ]:
def plot_comunas_data(sf, title, comunas, data=None, 
                      color=None, print_id=False):
    '''
    Plot map with selected comunes, using specific color
    '''
    
    color_ton, bins = calc_color(data, color)
    df = read_shapefile(sf)
    comuna_id = []
    shapes = sf.shapes()
    records= sf.records()
    for j in range(0, len(shapes)):
        record = sf.record(j)
        if record[8] == 'Greater Sydney':
            for i in comunas:
                comuna_id.append(df[df.SA2_NAME16 == i].index.get_values()[0])
    plot_map_fill_multiples_ids_tone(sf, title, comuna_id, 
                                     print_id, 
                                     color_ton, 
                                     bins, 
                                     x_lim = None, 
                                     y_lim = None, 
                                     figsize = (11,9));

In [ ]:
query = """
select area_id, area_name,  cyclability
from cyclability
"""
result= pgquery(conn, query,returntype='dict')
import pandas as pd
fileName = 'cyclability.csv'
data= pd.DataFrame(result)
data.to_csv(fileName,index = False)


In [ ]:
cyc = pd.read_csv('cyclability.csv')
title = 'cyclability Distrubution on Sydney Region'
data = cyc.cyclability
names = cyc.area_name


In [ ]:
%matplotlib notebook
plt.ion()
plot_comunas_data(sf, title, names, data, 4, True)
plt.show()


1: 0 => 0 
2: 1 => 1 
3: 2 => 1 
4: 2 => 2 
5: 3 => 3 
6: 4 => 14 

   1   2   3   4   5   6


### Machine Learning -- Confusion Matrix

##### JSON to CSV

In [ ]:
import requests
import pprint
import csv
response = requests.get("http://soit-app-pro-4.ucc.usyd.edu.au:3000/api/v1/json")

In [ ]:
data_extra = response.json()
header = data_extra[0]

newfile = []
filelen = len(data_extra)

# for j in range(filelen):
#     row = data_extra[j]
#     print(row[5])


for j in range(1,filelen):
    data_dic = {}
    row = data_extra[j]
    if len(row)>5:
        lat = row[5]
        lon = row[6]
        point = "POINT"+"("+str(lat)+" "+str(lon)+")"

        data_dic['OBJECTID'] = row[0]
        data_dic['Furniture_Symbology']=row[1]
        data_dic['Furniture_Type']=row[2]
        data_dic['Owners']=row[3]
        data_dic['point']=point
        data_dic['Bike_Parking_spaces']=row[4]
        data_dic['Easting']=row[7]
        data_dic['Northing']=row[8]
        newfile.append(data_dic)
        
# print(newfile[0])  
import pandas as pd
fileName = 'NewExtraData.csv'
number = 1
data= pd.DataFrame(newfile)

data.to_csv(fileName,index = False)


data1 = list(csv.DictReader(open('NewExtraData.csv')))


data_ED = list(csv.DictReader(open('NewExtraData.csv')))
pprint.pprint(data_ED[0])

##### Data Cleaning

In [ ]:
import csv
import pprint
import numpy as np
data_EXTRA = list(csv.DictReader(open('NewExtraData.csv')))
header_EXTRA = next(csv.reader(open('NewExtraData.csv')))

In [ ]:
filename = "NewExtraData.csv"
data = pd.read_csv(filename)

EXTRALEN = len(data_EXTRA)
newlist = []
for i in range(EXTRALEN):
    #print(type(data_EXTRA[i]['Bike_Parking_spaces']))
    try:
        if int(data_EXTRA[i]['Bike_Parking_spaces']):
            newlist.append(data_EXTRA[i])
    except:
        continue;
        
newfile = 'NewNewExtraData.csv'
data= pd.DataFrame(newlist)
data.to_csv(newfile,index = False)
data_BSPNEW= list(csv.DictReader(open('NewNewExtraData.csv')))
pprint.pprint(data_BSPNEW)

#### Schema for ExtraData

In [ ]:
# 1st: login to database
conn = pgconnect()

# if you want to reset the table
pgexec (conn, "DROP TABLE IF EXISTS ExtraData", None, "Reset Table ExtraData")

# 2nd: ensure that the schema is in place

ExtraData_schema = """CREATE TABLE IF NOT EXISTS ExtraData (
                         Bike_Parking_spaces   INTEGER,
                         Easting     NUMERIC ,
                         Furniture_Symbology  VARCHAR(100),
                         Furniture_Type  VARCHAR(100),
                         Northing NUMERIC,
                         Owners VARCHAR(100),
                         point GEOMETRY(point, 4326) ,
                         OBJECTID   INTEGER PRIMARY KEY
                        )"""
pgexec (conn, ExtraData_schema, None, "Create Table ExtraData")

# 3nd: load data
# IMPORTANT: make sure the header line of CSV is without spaces!
insert_stmt = """INSERT INTO ExtraData VALUES (%(Bike_Parking_spaces)s,%(Easting)s,%(Furniture_Symbology)s,%(Furniture_Type)s,%(Northing)s,%(Owners)s,ST_GEOMFROMTEXT(%(point)s, 4326),%(OBJECTID)s)"""


for row in data_BSPNEW:
    data_row = {}
    for i in row:
        data_row[i] = row[i]
    pgquery(conn, insert_stmt, args=data_row, msg="inserted ")

##### Combine the datasets (Extra + BSP) + Retrieve data: bikes & parking spaces

In [ ]:
query = """ 
select area_name, coalesce(AVG(Difference)::DECIMAL, 0.0) as bsascore from(
select area_name, num_bikes, Bike_Parking_spaces,num_bikes - Bike_Parking_spaces as Difference
        from neighbourhoodswithboundaries B left join BikeSharingPods BSP on (ST_Contains(B.boundaries,BSP.point)) 
        left join ExtraData as ED
        on ST_DWithin(BSP.point, ED.point, 5e-3)
        group by area_name,num_bikes, Bike_Parking_spaces) as A
 group by area_name  
"""
result= pgquery(conn, query,returntype='dict')
pprint.pprint(result)

#### Schema for BSAscore

In [ ]:
# 1st: login to database
conn = pgconnect()

# if you want to reset the table
pgexec (conn, "DROP TABLE IF EXISTS BSAscore", None, "Reset Table BSAscore")

# 2nd: ensure that the schema is in place

BSAscore_schema = """CREATE TABLE IF NOT EXISTS BSAscore(
                        area_name VARCHAR(100),
                          bsascore DECIMAL
                        )"""
pgexec (conn, BSAscore_schema, None, "Create Table BSAscore")

# 3nd: load data
# IMPORTANT: make sure the header line of CSV is without spaces!

insert_stmt = """INSERT INTO BSAscore VALUES (%(area_name)s, %(bsascore)s)"""

for row in result:
    temp_row = {}
    for i in row:
        temp_row[i] = row[i]
    pgquery(conn, insert_stmt, args=temp_row, msg="inserted ")

In [ ]:
query = """ 
select num_bikes as bikes, Bike_Parking_spaces as parkingspaces
        from BikeSharingPods as BSP
        join ExtraData as ED
        on ST_DWithin(BSP.point, ED.point, 5e-3)
        
"""
result= pgquery(conn, query,returntype='dict')
pprint.pprint(result)

##### Demo: Scatter plot -- parkingspaces vs bikes
- Initiate the use of the Confusion Matrix algorithm
- Not included in the actual analysis

In [ ]:
import matplotlib.pyplot as plt


y = np.arange(0,46)

plt.plot(y)
plt.plot(bikeslist, spacelist, 'bo')
plt.xlabel("Bikes")
plt.ylabel("Bike parking spaces")

plt.show()

##### Training Dataset + Test Dataset

In [ ]:
# The length of bikeslist and spacelist are exactly the same
whole_len = len(bikeslist)
test_len = int(0.5*whole_len)

testbikes = []
testspace = []

trainbikes = []
trainspace = []

for i in range(test_len):
    testbikes.append(bikeslist[i])
    testspace.append(spacelist[i])

trainbikes = bikeslist[test_len:-1]
trainspace = spacelist[test_len:-1]
#print(len(trainbikes))

In [ ]:
ltestbikes = [] 
ltrainbikes = []

ltestspace = []
ltrainspace = []

for e in testbikes:
    x= int(e)
    ltestbikes.append(x)
    
for r in trainbikes:
    y= int(r)
    ltrainbikes.append(y)
    
for se in testspace:
    z = int(se)
    ltestspace.append(z)

for sr in trainspace:
    w = int(sr)
    ltrainspace.append(w)

# print(len(ltestbikes))
# print(len(ltrainbikes))
# print(len(ltestspace))
# print(len(ltrainspace))

##### Data Aggregation: The difference
- BikeStoringAbility = Number of Bike parking spaces - Number of bikes



In [ ]:

LBSAtrain = []
for i in range(len(ltrainspace)):
    diff = ltrainspace[i] - ltrainbikes[i]
    LBSAtrain.append(diff)
    
LBSAtest = []
for i in range(len(ltestspace)):
    diff = ltestspace[i] - ltestbikes[i]
    LBSAtest.append(diff)
    
# print(len(LBSAtrain))
# print(len(LBSAtest))
# print(LBSAtrain)
# print("----")
# print(LBSAtest)

### Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
BSA_true = LBSAtrain
BSA_pred = LBSAtest
confusion_matrix(BSA_true, BSA_pred)

In [ ]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

data = {'BSA_Predicted': BSA_pred,
        'BSA_True':    BSA_true
        }

df = pd.DataFrame(data, columns=['BSA_True','BSA_Predicted'])
confusion_matrix = pd.crosstab(df['BSA_True'], df['BSA_Predicted'], rownames=['True'], colnames=['Predicted'], margins = True)

ax = plt.axes()
sn.heatmap(confusion_matrix, annot=True)

ax.set_title('The confusion matrix')
plt.show()

<div style="background-color:lightcyan;">
<span style="color:black"><center> <h1> Correlation Analysis </h1></center></span>
</div>

#### Schema for CyclabilityIncome

In [4]:
query = """ 
select area_id,area_name,median_annual_household_income as income,
            cyclability
        from cyclability natural join
        CensusStats
"""
result= pgquery(conn, query,returntype='dict')
pprint.pprint(result)

NameError: name 'pgquery' is not defined

In [ ]:
conn = pgconnect()

# if you want to reset the table
pgexec (conn, "DROP TABLE IF EXISTS CyclabilityIncome", None, "Reset Table CyclabilityIncome")

# 2nd: ensure that the schema is in place
CyclabilityIncome_schema = """CREATE TABLE IF NOT EXISTS CyclabilityIncome (
                         area_id   INTEGER PRIMARY KEY,
                         area_name     VARCHAR(100),
                         income NUMERIC,
                         cyclability DECIMAl
                        )"""
pgexec (conn, CyclabilityIncome_schema, None, "Create Table CyclabilityIncome")

# 3nd: load data
# IMPORTANT: make sure the header line of CSV is without spaces!
insert_stmt = """INSERT INTO CyclabilityIncome VALUES (%(area_id)s,%(area_name)s,%(income)s,%(cyclability)s)"""

for row in result:
    temp_row = {}
    for i in row:
        temp_row[i] = row[i]
    pgquery(conn, insert_stmt, args=temp_row, msg="inserted ")

#### Schema for CyclabilityRent

In [ ]:
query = """ 
select area_id,area_name,avg_monthly_rent as rent,
            cyclability
        from cyclability natural join
        CensusStats
"""
result= pgquery(conn, query,returntype='dict')
pprint.pprint(result)

In [ ]:
conn = pgconnect()

# if you want to reset the table
pgexec (conn, "DROP TABLE IF EXISTS CyclabilityRent", None, "Reset Table CyclabilityRent")

# 2nd: ensure that the schema is in place
CyclabilityRent_schema = """CREATE TABLE IF NOT EXISTS CyclabilityRent (
                         area_id   INTEGER PRIMARY KEY,
                         area_name     VARCHAR(100),
                         rent NUMERIC,
                         cyclability DECIMAl
                        )"""
pgexec (conn, CyclabilityRent_schema, None, "Create Table CyclabilityRent")

# 3nd: load data
# IMPORTANT: make sure the header line of CSV is without spaces!
insert_stmt = """INSERT INTO CyclabilityRent VALUES (%(area_id)s,%(area_name)s,%(rent)s,%(cyclability)s)"""

for row in result:
    temp_row = {}
    for i in row:
        temp_row[i] = row[i]
    pgquery(conn, insert_stmt, args=temp_row, msg="inserted ")

### Scatter plots: the correlations

#### Cyclability vs Income

In [ ]:
query = """
select cyclability, income
from CyclabilityIncome
"""
result= pgquery(conn, query,returntype='list')
# pprint.pprint(result)

cyclist = []
incomelist = []

incomecyc_len = len(result)
for i in range(incomecyc_len):
    cyclability = result[i][0]
    cyclist.append(cyclability)
    income = result[i][1]
    incomelist.append(income)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(incomelist, cyclist, 'bo')
plt.xlabel("Median annual household income")
plt.ylabel("Cyclability Score")

X = incomelist
Y = cyclist

# solve for a and b
def best_fit(X, Y):

    xbar = sum(X)/len(X)
    ybar = sum(Y)/len(Y)
    n = len(X)

    numerator = sum([xi*yi for xi,yi in zip(X, Y)]) - n * xbar * ybar
    denominator = sum([xi**2 for xi in X]) - n * xbar**2

    b = numerator / denominator
    a = ybar - b * xbar

    print('y = {:.2f}x + {:.2f}'.format(b, a))

    return a, b

a, b = best_fit(X, Y)
yfit = [a + b * xi for xi in X]
plt.plot(X, yfit)

plt.show()

#### Cyclability vs Rent

In [ ]:
query = """
select cyclability, rent
from CyclabilityRent
"""
result= pgquery(conn, query,returntype='list')
pprint.pprint(result)

cyclist = []
rentlist = []

rentcyc_len = len(result)
for i in range(incomecyc_len):
    cyclability = result[i][0]
    cyclist.append(cyclability)
    rent = result[i][1]
    rentlist.append(rent)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(rentlist, cyclist, 'ro')
plt.xlabel("Average monthly rent")
plt.ylabel("Cyclability Score")
plt.title("Cyclability score vs Average monthly rent")

X = rentlist
Y = cyclist

a, b = best_fit(X, Y)
yfit = [a + b * xi for xi in X]
plt.plot(X, yfit)

plt.show()

<div style="background-color:lightcyan;">
<span style="color:black"><center> <h1> The END of the Analysis </h1></center></span>
</div>